In [ ]:
!pip install gpytorch
!pip install torcheval

In [2]:
import math
import torch
import gpytorch
from torcheval.metrics import R2Score
import pandas as pd
from matplotlib import pyplot as plt
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
#чтение данных
with open('path_to_csv', 'rb') as file:
  df = pd.read_csv(file)

In [ ]:
#вычисление дополнительных спектральных индексов
df['sr4_3_mean'] = df['B08_mean']/df['B04_mean']
df['sr5_4_mean'] = df['B11_mean']/df['B08_mean']
df['kt_1_mean'] = 0.3037 * df['B02_mean'] + 0.2793 * df['B03_mean'] + 0.4743 * df['B04_mean'] + 0.5585 * df['B08_mean'] + 0.5082 * df['B11_mean'] + 0.1863 * df['B12_mean']
df['kt_2_mean'] = - 0.2941 * df['B02_mean'] - 0.243 * df['B03_mean'] - 0.5424 * df['B04_mean'] + 0.7276 * df['B08_mean'] + 0.0713 * df['B11_mean'] - 0.1608 * df['B12_mean']
df['kt_3_mean'] = 0.1511 * df['B02_mean'] + 0.1973 * df['B03_mean'] + 0.3283 * df['B04_mean'] + 0.3407 * df['B08_mean'] - 0.7117 * df['B11_mean'] - 0.4559 * df['B12_mean']

In [ ]:
#вычисление усредненных географичеких координат выделов
def avg_long(polygon):
  my_string = polygon
  my_string = re.sub(r'[A-Za-z()]', '', my_string)
  mean_longitude = 0
  с = 0
  for i in range(len(my_string.split(','))):
    mean_longitude += float(my_string.split(',')[i].split(' ')[1])
    c = i + 1
  mean_longitude = mean_longitude / c
  return mean_longitude

def avg_lat(polygon):
  my_string = polygon
  my_string = re.sub(r'[A-Za-z()]', '', my_string)
  mean_latitude = 0
  с = 0
  for i in range(len(my_string.split(','))):
    mean_latitude += float(my_string.split(',')[i].split(' ')[2])
    c = i + 1
  mean_latitude = mean_latitude / c
  return mean_latitude

df['mean_latitude'] = df['geometry'].apply(avg_lat)
df['mean_longitude'] = df['geometry'].apply(avg_long)

In [ ]:
#выделение целевых значений
target_1 = df['height']
target_2 = df['timber_stock']
target_3 = df['basal_area']
df.drop(['date', 'height', 'timber_stock', 'basal_area', 'geometry'], axis= 1, inplace=True)
#масштабирование данных
scaler = StandardScaler()
scaler.fit(df)
data = scaler.transform(df)

data = df.to_numpy()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
x_data = torch.tensor(data, dtype=torch.float32).to(device)

y_data = torch.stack([
    torch.tensor(target_1.to_numpy(), dtype=torch.float32),
    torch.tensor(target_2.to_numpy(), dtype=torch.float32),
    torch.tensor(target_3.to_numpy(), dtype=torch.float32),
], -1).to(device)
#разделение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)

In [ ]:
#модель Multi-Task Gaussian Process Regression
class MultitaskGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.ConstantMean(), num_tasks=3
        )
        self.covar_module = gpytorch.kernels.MultitaskKernel(
            gpytorch.kernels.RBFKernel(), num_tasks=3, rank=2
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)


likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=3)
model = MultitaskGPModel(X_train, y_train, likelihood)

In [ ]:
import os
smoke_test = ('CI' in os.environ)
training_iterations = 2 if smoke_test else 100

#поиск оптимальных гиперпараметров модели
model.train().to(device)
likelihood.train()
history = []

optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

#loss для GPs - предельное логарифмическое правдоподобие
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iterations):
    optimizer.zero_grad()
    output = model(X_train)
    loss = -mll(output, y_train)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f' % (i + 1, training_iterations, loss.item()))
    optimizer.step()
    history.append(loss.item())

In [ ]:
#график обучения
plt.plot(history)
plt.xlabel('epoch')
plt.ylabel('the marginal log likelihood (loss)')
plt.title('Training MTGPR')
plt.show()

In [ ]:
model.eval()
likelihood.eval()

#предсказания модели
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    predictions = likelihood(model(X_test))
    mean = predictions.mean
    lower, upper = predictions.confidence_region()

In [ ]:
#подсчет метрик результатов
metric_1 = R2Score().update(mean[:, 0], y_test[:, 0])
r_sq_1 = metric_1.compute()
mse_1 = torch.nn.MSELoss()(mean[:, 0], y_test[:, 0])
metric_2 = R2Score().update(mean[:, 1], y_test[:, 1])
r_sq_2 = metric_2.compute()
mse_2 = torch.nn.MSELoss()(mean[:, 1], y_test[:, 1])
metric_3 = R2Score().update(mean[:, 2], y_test[:, 2])
r_sq_3 = metric_3.compute()
mse_3 = torch.nn.MSELoss()(mean[:, 2], y_test[:, 2])
print(f"coefficient of determination (height): {r_sq_1}")
print("MSE (height): {}".format(mse_1))
print(f"coefficient of determination (timber_stock): {r_sq_2}")
print("MSE (timber_stock): {}".format(mse_2))
print(f"coefficient of determination (basal_area): {r_sq_3}")
print("MSE (basal_area): {}".format(mse_3))